## Proprocess raw texts to digital indices and train them to word embeddings via gensim's word2vec

**Libraries used:**
1. nltk: used to tokenize text

2. gensim: use its models.word2vec to produce word vectors.

**Note: This is summerized in a more concise python file Wemb_gensim.py **

In [1]:
import os
import glob
import gensim, logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import nltk
import numpy
import pickle as pkl
import sys
import multiprocessing

In [2]:
# Strip punctuation/symbols from words
def normalize_text(text):
    norm_text = text.lower()
    
    # Replace breaks with spaces
    norm_text = norm_text.replace('<br />', ' ')

    # Pad punctuation with spaces on both sides
    for char in [':', '"', ',', '(', ')', '!', '?', ';', '*']:
        norm_text = norm_text.replace(char, ' ')

    norm_text = norm_text.replace('.', ' ' + '.')
    return norm_text

In [5]:
def read_dataset(path):
    if sys.version > '3':
        control_chars = [chr(0x85)] # UTF-8
    else:
        control_chars = [unichr(0x85)] # UTF-16 which is often used in other language codes (e.g. Chinese)

    dataset = []
    currdir = os.getcwd()
    os.chdir(path)
    #i = 0
    for ff in glob.glob("*.txt"):
        #i += 1
        with open(ff, "r") as f:
            line_txt = f.readline().strip()
            line_norm = normalize_text(line_txt)
            dataset.append(line_norm)
        #if(i==100):
            #break
    os.chdir(currdir)
    return dataset

In [9]:
def build_dict(path, Wemb_size=128, iter=10):
    cores = multiprocessing.cpu_count()
    assert gensim.models.doc2vec.FAST_VERSION > -1, "This will be painfully slow otherwise"

    sentences_pos = read_dataset(path+'/pos/')
    sentences_neg = read_dataset(path+'/neg/')

    sentences_train = sentences_pos + sentences_neg
    tokenized_sentences = [nltk.word_tokenize(sent) for sent in sentences_train]

    model = gensim.models.Word2Vec(tokenized_sentences, min_count=1,
                                   size=Wemb_size, window=5,
                                   workers=cores, iter=iter)

    tok_sents_pos = tokenized_sentences[:len(sentences_pos)]
    tok_sents_neg = tokenized_sentences[len(sentences_pos):]

    return {'model': model, 'tok_sents_pos': tok_sents_pos, 'tok_sents_neg': tok_sents_neg}

### Convert words in text sentences to their corresponding indices in the dictionary

In [10]:
def sentence2idx(tokenized_sentences, model):
    idx = []
    for tok_sen in tokenized_sentences:
        idx_sent = numpy.zeros(len(tok_sen), dtype=numpy.int)
        for i, word in enumerate(tok_sen):
            if word in model.wv.vocab:
                idx_sent[i] = model.wv.vocab[word].index
            else:
                idx_sent[i] = model.wv.vocab['.'].index
        idx.append(idx_sent)
    return idx

### Convert words in the dictionary into embeddings

Actually, there is no need to create word embedding specifically, because it is included in the trained gensim model and can be acessed by `model.wv.syn0`.

In [11]:
def create_Wemb(model, Wemb_size=128):
    Wemb = numpy.zeros((len(model.wv.vocab), Wemb_size))
    for i in range(len(model.wv.vocab)):
        embedding_vector = model.wv[model.wv.index2word[i]]
        if embedding_vector is not None:
            Wemb[i] = embedding_vector
    return Wemb

In [14]:
data_path = '/Users/lifa08/Local_documents/Machine_Learning/Miniproject_test/aclImdb/train'

In [15]:
result = build_dict(data_path)

2018-07-02 17:18:52,626 : INFO : collecting all words and their counts
2018-07-02 17:18:52,627 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-07-02 17:18:53,139 : INFO : PROGRESS: at sentence #10000, processed 2518483 words, keeping 63047 word types
2018-07-02 17:18:53,724 : INFO : PROGRESS: at sentence #20000, processed 5010758 words, keeping 90480 word types
2018-07-02 17:18:54,034 : INFO : collected 101119 word types from a corpus of 6251170 raw words and 25000 sentences
2018-07-02 17:18:54,035 : INFO : Loading a fresh vocabulary
2018-07-02 17:18:54,576 : INFO : min_count=1 retains 101119 unique words (100% of original 101119, drops 0)
2018-07-02 17:18:54,579 : INFO : min_count=1 leaves 6251170 word corpus (100% of original 6251170, drops 0)
2018-07-02 17:18:54,899 : INFO : deleting the raw counts dictionary of 101119 items
2018-07-02 17:18:54,904 : INFO : sample=0.001 downsamples 48 most-common words
2018-07-02 17:18:54,905 : INFO : downsampling lea

2018-07-02 17:20:01,231 : INFO : training on 62511700 raw words (45593202 effective words) took 63.9s, 713137 effective words/s


In [16]:
sents_pos = result['tok_sents_pos']
model = result['model']
train_x_pos = sentence2idx(sents_pos, model)

In [17]:
model.wv.most_similar('good')

2018-07-02 17:20:10,797 : INFO : precomputing L2-norms of word weight vectors


[('decent', 0.7537664771080017),
 ('great', 0.6990833282470703),
 ('bad', 0.6676902174949646),
 ('cool', 0.6532009840011597),
 ('lousy', 0.6135501265525818),
 ('nice', 0.6117060780525208),
 ('solid', 0.6004713773727417),
 ('fine', 0.591917872428894),
 ('ok', 0.5889630317687988),
 ('terrific', 0.5840588808059692)]

In [22]:
sents_neg = result['tok_sents_neg']
train_x_neg = sentence2idx(sents_neg, model)
print(train_x_neg[:2])

[array([  792,    16,    29,     4,     0,   118,  1764,  7121,    10,
          19,   983,     5,    28, 20422,     5,     7,    12,  2375,
        1807,   128,  2230,     5,     3,  6987,   300,     1,  2584,
        2314,     0,    19,    36,   484,  5942,    12,  3371,     2,
          39,    12,     3,  1003,   175,    21,    50,   782,     1]), array([   78,     1,     1, 49001,     9,     0,   197,   627,   131,
           8,  3345,     2,     9,   250,     0,    17,   179,   710,
           5,   107,     1,   190,    32, 12560,     5,    99, 10283,
        1478,     2,  3369,     1,  1642,    71,   147,   103,   627,
         168,  4411,   204,   102,    32,  1747,     0,    90,    29,
           9,   375,  2383, 26621,   151,    63,   537,   305,     6,
        2200,     4,  9892,     0,   861,   137,     8,  6074,   397,
          55,    32,   955,   155,    29,     4,   132, 19296, 12672,
           4,     3,    17, 10283,  5509,    61,    23,    28,  1148,
           3,    

In [19]:
train_x = train_x_pos + train_x_neg

In [21]:
train_y = [1] * len(train_x_pos) + [0] * len(train_x_neg)
# print(train_y)

In [23]:
test_path = '/Users/lifa08/Local_documents/Machine_Learning/Miniproject_test/aclImdb/test'
test_sents_pos = read_dataset(test_path+'/pos/')

In [24]:
tok_test_sents_pos = [nltk.word_tokenize(sent) for sent in test_sents_pos]
test_x_pos = sentence2idx(tok_test_sents_pos, model)

In [25]:
test_sents_neg = read_dataset(test_path+'/neg/')
tok_test_sents_neg = [nltk.word_tokenize(sent) for sent in test_sents_neg]
test_x_neg = sentence2idx(tok_test_sents_neg, model)
test_x = test_x_pos + test_x_neg
test_y = [1] * len(test_x_pos) + [0] * len(test_x_neg)

### Save converted sentence indices to a file

In [26]:
f = open('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/gensim_imdb.pkl', 'wb')
pkl.dump((train_x, train_y), f, -1)
pkl.dump((test_x, test_y), f, -1)
f.close()

In [30]:
f = open('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/gensim_imdb.pkl', 'rb')
train_set = pkl.load(f)
test_set = pkl.load(f)
f.close()

### Save word embeddings to a file

In [36]:
Wemb = create_Wemb(model)

In [34]:
f = open('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/gensim_imdb_Wemb.pkl', 'wb')
pkl.dump(Wemb, f, -1)
f.close()

In [38]:
f = open('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/gensim_imdb_Wemb.pkl', 'rb')
Wemb_file = pkl.load(f)
f.close()

In [39]:
print(Wemb)

[[  5.98381102e-01   4.73410547e-01   1.33979070e+00 ...,  -1.87317252e+00
    2.34784722e+00   8.82582009e-01]
 [ -6.02306604e-01  -3.29882771e-01   1.81562877e+00 ...,  -1.46761751e+00
    1.23798764e+00   8.82097363e-01]
 [ -6.25977695e-01  -6.34995878e-01   1.92042744e+00 ...,  -8.90727460e-01
    3.45125347e-02   8.86263549e-01]
 ..., 
 [  5.45666972e-03  -1.41647756e-02   5.51845878e-03 ...,   4.92366478e-02
   -1.74962711e-02   1.49816228e-02]
 [ -1.06746666e-02  -2.27900594e-03   1.81603748e-02 ...,  -3.55963185e-02
    2.40580682e-02   1.41165266e-03]
 [  5.12123890e-02   3.75586897e-02  -4.52179313e-02 ...,   3.36575974e-03
   -4.37835837e-03   4.71857414e-02]]


In [40]:
print(Wemb_file)

[[  5.98381102e-01   4.73410547e-01   1.33979070e+00 ...,  -1.87317252e+00
    2.34784722e+00   8.82582009e-01]
 [ -6.02306604e-01  -3.29882771e-01   1.81562877e+00 ...,  -1.46761751e+00
    1.23798764e+00   8.82097363e-01]
 [ -6.25977695e-01  -6.34995878e-01   1.92042744e+00 ...,  -8.90727460e-01
    3.45125347e-02   8.86263549e-01]
 ..., 
 [  5.45666972e-03  -1.41647756e-02   5.51845878e-03 ...,   4.92366478e-02
   -1.74962711e-02   1.49816228e-02]
 [ -1.06746666e-02  -2.27900594e-03   1.81603748e-02 ...,  -3.55963185e-02
    2.40580682e-02   1.41165266e-03]
 [  5.12123890e-02   3.75586897e-02  -4.52179313e-02 ...,   3.36575974e-03
   -4.37835837e-03   4.71857414e-02]]


### Include all above into a function

In [48]:
def train_gensim_w2vec(path):
    # path = '/Users/lifa08/Local_documents/Machine_Learning/Miniproject_test/aclImdb/'
    result = build_dict(path+'train')
    model = result['model']

    sents_pos = result['tok_sents_pos']
    train_x_pos = sentence2idx(sents_pos, model)

    sents_neg = result['tok_sents_neg']
    train_x_neg = sentence2idx(sents_neg, model)
    train_x = train_x_pos + train_x_neg
    train_y = [1] * len(train_x_pos) + [0] * len(train_x_neg)

    test_sents_pos = read_dataset(path+'test/pos/')
    tok_test_sents_pos = [nltk.word_tokenize(sent) for sent in test_sents_pos]
    test_x_pos = sentence2idx(tok_test_sents_pos, model)

    test_sents_neg = read_dataset(path+'test/neg/')
    tok_test_sents_neg = [nltk.word_tokenize(sent) for sent in test_sents_neg]
    test_x_neg = sentence2idx(tok_test_sents_neg, model)
    test_x = test_x_pos + test_x_neg
    test_y = [1] * len(test_x_pos) + [0] * len(test_x_neg)

    f = open('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/gensim_imdb.pkl', 'wb')
    pkl.dump((train_x, train_y), f, -1)
    pkl.dump((test_x, test_y), f, -1)
    f.close()

    Wemb = create_Wemb(model)
    f = open('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/gensim_imdb_Wemb.pkl', 'wb')
    pkl.dump(Wemb, f, -1)
    f.close()

    model.save('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel')

In [49]:
path = '/Users/lifa08/Local_documents/Machine_Learning/Miniproject_test/aclImdb/'
train_gensim_w2vec(path)

2018-07-02 18:09:32,880 : INFO : collecting all words and their counts
2018-07-02 18:09:32,882 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2018-07-02 18:09:33,363 : INFO : PROGRESS: at sentence #10000, processed 2518483 words, keeping 63047 word types
2018-07-02 18:09:33,897 : INFO : PROGRESS: at sentence #20000, processed 5010758 words, keeping 90480 word types
2018-07-02 18:09:34,198 : INFO : collected 101119 word types from a corpus of 6251170 raw words and 25000 sentences
2018-07-02 18:09:34,199 : INFO : Loading a fresh vocabulary
2018-07-02 18:09:35,913 : INFO : min_count=1 retains 101119 unique words (100% of original 101119, drops 0)
2018-07-02 18:09:35,914 : INFO : min_count=1 leaves 6251170 word corpus (100% of original 6251170, drops 0)
2018-07-02 18:09:36,197 : INFO : deleting the raw counts dictionary of 101119 items
2018-07-02 18:09:36,201 : INFO : sample=0.001 downsamples 48 most-common words
2018-07-02 18:09:36,202 : INFO : downsampling lea

2018-07-02 18:10:44,390 : INFO : worker thread finished; awaiting finish of 2 more threads
2018-07-02 18:10:44,392 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-07-02 18:10:44,404 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-07-02 18:10:44,405 : INFO : training on 62511700 raw words (45599438 effective words) took 66.5s, 685607 effective words/s
2018-07-02 18:12:07,201 : INFO : saving Word2Vec object under /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel, separately None
2018-07-02 18:12:07,202 : INFO : storing np array 'syn0' to /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel.wv.syn0.npy
2018-07-02 18:12:07,400 : INFO : not storing attribute syn0norm
2018-07-02 18:12:07,404 : INFO : storing np array 'syn1neg' to /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel.syn1neg.npy
2018-07-02 18:12:07,578 : INFO : not storing att

### Check if embeddings from gensim model equal to the word embeddings stored in the file to make sure the correctness of storing word embeddings.

In [56]:
def compare_idxwemb_wordemb(sentence_idx):
    model_path = '/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel'
    model = gensim.models.Word2Vec.load(model_path)
    sentence_words = []
    for idx, x in enumerate(sentence_idx):
        sentence_words.append(model.wv[model.wv.index2word[x]])

    f = open('/Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/gensim_imdb_Wemb.pkl', 'rb')
    Wemb = pkl.load(f)
    f.close()
    emb_x = Wemb[sentence_idx]

    return (numpy.matrix(sentence_words)==numpy.matrix(emb_x)).all()

In [57]:
print(train_x_neg[0])
compare_idxwemb_wordemb(train_x_neg[0])

2018-07-02 18:24:33,180 : INFO : loading Word2Vec object from /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel


[  792    16    29     4     0   118  1764  7121    10    19   983     5
    28 20422     5     7    12  2375  1807   128  2230     5     3  6987
   300     1  2584  2314     0    19    36   484  5942    12  3371     2
    39    12     3  1003   175    21    50   782     1]


2018-07-02 18:24:33,418 : INFO : loading wv recursively from /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel.wv.* with mmap=None
2018-07-02 18:24:33,420 : INFO : loading syn0 from /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel.wv.syn0.npy with mmap=None
2018-07-02 18:24:33,458 : INFO : setting ignored attribute syn0norm to None
2018-07-02 18:24:33,461 : INFO : loading syn1neg from /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel.syn1neg.npy with mmap=None
2018-07-02 18:24:33,507 : INFO : setting ignored attribute cum_table to None
2018-07-02 18:24:33,511 : INFO : loaded /Users/lifa08/Local_documents/Machine_Learning/miniproject/gensim/imdb_gensim_w2vmodel


True